In [1]:
import os
os.getcwd()
%cd EDTalk/

/home/ldap-users-2/lathifgalih-k/clone-video/edtalk-exp/EDTalk


In [2]:
import sys
print(sys.executable)

/home/is/lathifgalih-k/miniconda3/envs/EDTalk/bin/python


#### Preprocess Data Video n Generation Video

In [1]:
import os
import subprocess
from pathlib import Path
import shutil
from tqdm import tqdm
import yaml
import random

# Configuration
BASE_DIR = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video"
MUSE_AUDIO_DIR = f"{BASE_DIR}/muse-exp/MuseTalk/data/audio" 
AVATAR_IMAGE_DIR = f"{BASE_DIR}/avatar-cloned/image-editing/output_image"
EDTALK_DIR = f"{BASE_DIR}/edtalk-exp/EDTalk"
OUTPUT_DIR = f"{EDTALK_DIR}/results"

def run_edtalk_avatar_inference(source_img, audio_path, output_path):
    """Run EDTalk inference with avatar image as source"""
    cmd = f'python demo_lip_pose.py --fix_pose --source_path "{source_img}" --audio_driving_path "{audio_path}" --save_path "{output_path}" --face_sr'
    result = subprocess.run(cmd, shell=True, capture_output=True, cwd=EDTALK_DIR)
    
    # Print error output for debugging
    if result.returncode != 0:
        print(f"Error output: {result.stderr.decode() if result.stderr else 'No error message'}")
        print(f"Stdout: {result.stdout.decode() if result.stdout else 'No stdout'}")
    
    return result.returncode == 0

def get_random_avatar_image(subject_id, subject_type):
    """Get a random avatar image for the subject"""
    avatar_folder = f"{AVATAR_IMAGE_DIR}/{subject_type}/{subject_id}"
    
    if not os.path.exists(avatar_folder):
        print(f"Avatar folder not found: {avatar_folder}")
        return None
    
    # Get all avatar images (excluding grid image)
    avatar_files = [f for f in os.listdir(avatar_folder) 
                   if f.endswith('.png') and not f.endswith('_grid.png')]
    
    if not avatar_files:
        print(f"No avatar images found in: {avatar_folder}")
        return None
    
    # Randomly select one avatar image
    selected_avatar = random.choice(avatar_files)
    avatar_path = os.path.join(avatar_folder, selected_avatar)
    
    print(f"Selected avatar for {subject_id}: {selected_avatar}")
    return avatar_path

def get_audio_type_from_path(audio_path):
    """Extract audio type from path"""
    if "CosyVoice2" in audio_path:
        return "CV"
    elif "XTTSv2" in audio_path:
        return "XT"
    elif "YourTTS" in audio_path:
        return "YT"
    return "Unknown"

def get_dataset_type_from_path(audio_path):
    """Extract dataset type from path"""
    if "VCTK" in audio_path:
        return "VCTK"
    elif "Raw" in audio_path:
        return "Raw"
    return "Unknown"

def get_subject_task_from_audio(audio_path):
    """Extract subject and task from audio filename"""
    filename = os.path.basename(audio_path)
    # Remove prefix (CV_, XT_, YT_) and suffix (_cloned.wav)
    parts = filename.replace("CV_", "").replace("XT_", "").replace("YT_", "").replace("_cloned.wav", "")
    
    if "_VCTK" in parts:
        parts = parts.replace("_VCTK", "")
    
    return parts

def create_output_filename_avatar(subject_id, audio_path, avatar_filename):
    """Create output filename based on subject, audio, and avatar"""
    audio_type = get_audio_type_from_path(audio_path)
    subject_task = get_subject_task_from_audio(audio_path)
    avatar_id = Path(avatar_filename).stem  # e.g., A002_avatar_00
    
    return f"{avatar_id}_{audio_type}_{subject_task}_cloned.mp4"

def scan_and_process_avatar_files():
    """Scan available avatar images and audio files, then process them"""
    
    # Create output directories
    os.makedirs(f"{OUTPUT_DIR}/test_raw/avatar/RAW", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/test_vctk/avatar/VCTK", exist_ok=True)
    
    # Scan available audio folders
    audio_base_folders = [
        "ALS-Raw", "ALS-VCTK", "Stroke-Raw", "Stroke-VCTK"
    ]
    
    all_tasks = []
    
    # Scan avatar directories
    subject_types = ["ALS", "Stroke"]
    
    for subject_type in subject_types:
        avatar_type_dir = f"{AVATAR_IMAGE_DIR}/{subject_type}"
        if not os.path.exists(avatar_type_dir):
            continue
        
        # Get all subject directories
        for subject_id in os.listdir(avatar_type_dir):
            subject_path = os.path.join(avatar_type_dir, subject_id)
            if not os.path.isdir(subject_path):
                continue
            
            # Get random avatar image for this subject
            avatar_image_path = get_random_avatar_image(subject_id, subject_type)
            if not avatar_image_path:
                continue
            
            # Find corresponding audio files
            for audio_base_folder in audio_base_folders:
                if not (subject_type in audio_base_folder):
                    continue
                    
                audio_folder_path = f"{MUSE_AUDIO_DIR}/{audio_base_folder}"
                if not os.path.exists(audio_folder_path):
                    continue
                
                # Check each audio type folder
                for audio_type_folder in ["CosyVoice2", "XTTSv2", "YourTTS"]:
                    audio_type_path = f"{audio_folder_path}/{audio_type_folder}"
                    if not os.path.exists(audio_type_path):
                        continue
                    
                    # Find matching audio files
                    for audio_file in os.listdir(audio_type_path):
                        if not audio_file.endswith('.wav'):
                            continue
                        
                        # Check if audio file matches the subject
                        if subject_id in audio_file:
                            audio_path = os.path.join(audio_type_path, audio_file)
                            
                            task = {
                                "avatar_path": avatar_image_path,
                                "audio_path": audio_path,
                                "subject_id": subject_id,
                                "subject_type": subject_type,
                                "dataset_type": "Raw" if "Raw" in audio_base_folder else "VCTK",
                                "avatar_filename": os.path.basename(avatar_image_path)
                            }
                            all_tasks.append(task)
    
    print(f"Found {len(all_tasks)} avatar-audio pairs to process")
    
    # Process all tasks
    for task in tqdm(all_tasks, desc="Processing all avatar tasks"):
        avatar_path = task["avatar_path"]
        audio_path = task["audio_path"]
        subject_id = task["subject_id"]
        subject_type = task["subject_type"]
        dataset_type = task["dataset_type"]
        avatar_filename = task["avatar_filename"]
        
        # Create output filename
        output_filename = create_output_filename_avatar(subject_id, audio_path, avatar_filename)
        
        if dataset_type == "Raw":
            output_path = f"{OUTPUT_DIR}/test_raw/avatar/RAW/{output_filename}"
        else:  # VCTK
            output_path = f"{OUTPUT_DIR}/test_vctk/avatar/VCTK/{output_filename}"
        
        # Skip if output already exists
        if os.path.exists(output_path):
            print(f"Skipping existing: {output_filename}")
            continue
        
        # Check if avatar image exists
        if not os.path.exists(avatar_path):
            print(f"Avatar image not found: {avatar_path}")
            continue
        
        # Check if audio file exists
        if not os.path.exists(audio_path):
            print(f"Audio file not found: {audio_path}")
            continue
        
        # Use avatar image directly without cropping
        source_img = avatar_path
        
        # Run EDTalk inference
        print(f"Processing: {output_filename}")
        print(f"Avatar: {os.path.basename(avatar_path)}")
        print(f"Audio: {os.path.basename(audio_path)}")
        
        success = run_edtalk_avatar_inference(source_img, audio_path, output_path)
        
        if success:
            print(f"✓ Generated: {output_path}")
        else:
            print(f"✗ Failed: {output_path}")

def scan_and_process_avatar_files_with_pose():
    """Scan available avatar images and process them with pose from original videos"""
    
    # Create output directories
    os.makedirs(f"{OUTPUT_DIR}/test_raw/avatar_pose/RAW", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/test_vctk/avatar_pose/VCTK", exist_ok=True)
    
    # Video directories for pose reference
    MUSE_VIDEO_DIR = f"{BASE_DIR}/muse-exp/MuseTalk/data/video"
    video_folders = {
        "als-input": f"{MUSE_VIDEO_DIR}/als-input",
        "stroke-input": f"{MUSE_VIDEO_DIR}/stroke-input"
    }
    
    # Scan available audio folders
    audio_base_folders = [
        "ALS-Raw", "ALS-VCTK", "Stroke-Raw", "Stroke-VCTK"
    ]
    
    all_tasks = []
    
    # Scan avatar directories
    subject_types = ["ALS", "Stroke"]
    
    for subject_type in subject_types:
        avatar_type_dir = f"{AVATAR_IMAGE_DIR}/{subject_type}"
        if not os.path.exists(avatar_type_dir):
            continue
        
        # Get all subject directories
        for subject_id in os.listdir(avatar_type_dir):
            subject_path = os.path.join(avatar_type_dir, subject_id)
            if not os.path.isdir(subject_path):
                continue
            
            # Get random avatar image for this subject
            avatar_image_path = get_random_avatar_image(subject_id, subject_type)
            if not avatar_image_path:
                continue
            
            # Find corresponding video for pose reference
            video_folder = video_folders["als-input"] if subject_type == "ALS" else video_folders["stroke-input"]
            pose_video_path = None
            
            if os.path.exists(video_folder):
                for video_file in os.listdir(video_folder):
                    if video_file.startswith(subject_id) and video_file.endswith('.mp4'):
                        pose_video_path = os.path.join(video_folder, video_file)
                        break
            
            if not pose_video_path:
                print(f"No pose video found for {subject_id}, skipping pose generation")
                continue
            
            # Find corresponding audio files
            for audio_base_folder in audio_base_folders:
                if not (subject_type in audio_base_folder):
                    continue
                    
                audio_folder_path = f"{MUSE_AUDIO_DIR}/{audio_base_folder}"
                if not os.path.exists(audio_folder_path):
                    continue
                
                # Check each audio type folder
                for audio_type_folder in ["CosyVoice2", "XTTSv2", "YourTTS"]:
                    audio_type_path = f"{audio_folder_path}/{audio_type_folder}"
                    if not os.path.exists(audio_type_path):
                        continue
                    
                    # Find matching audio files
                    for audio_file in os.listdir(audio_type_path):
                        if not audio_file.endswith('.wav'):
                            continue
                        
                        # Check if audio file matches the subject
                        if subject_id in audio_file:
                            audio_path = os.path.join(audio_type_path, audio_file)
                            
                            task = {
                                "avatar_path": avatar_image_path,
                                "audio_path": audio_path,
                                "pose_video_path": pose_video_path,
                                "subject_id": subject_id,
                                "subject_type": subject_type,
                                "dataset_type": "Raw" if "Raw" in audio_base_folder else "VCTK",
                                "avatar_filename": os.path.basename(avatar_image_path)
                            }
                            all_tasks.append(task)
    
    print(f"Found {len(all_tasks)} avatar-audio-pose pairs to process")
    
    # Process all tasks
    for task in tqdm(all_tasks, desc="Processing all avatar pose tasks"):
        avatar_path = task["avatar_path"]
        audio_path = task["audio_path"]
        pose_video_path = task["pose_video_path"]
        subject_id = task["subject_id"]
        subject_type = task["subject_type"]
        dataset_type = task["dataset_type"]
        avatar_filename = task["avatar_filename"]
        
        # Create output filename
        output_filename = create_output_filename_avatar(subject_id, audio_path, avatar_filename)
        
        if dataset_type == "Raw":
            output_path = f"{OUTPUT_DIR}/test_raw/avatar_pose/RAW/{output_filename}"
        else:  # VCTK
            output_path = f"{OUTPUT_DIR}/test_vctk/avatar_pose/VCTK/{output_filename}"
        
        # Skip if output already exists
        if os.path.exists(output_path):
            print(f"Skipping existing: {output_filename}")
            continue
        
        # Check if files exist
        if not os.path.exists(avatar_path):
            print(f"Avatar image not found: {avatar_path}")
            continue
        
        if not os.path.exists(audio_path):
            print(f"Audio file not found: {audio_path}")
            continue
            
        if not os.path.exists(pose_video_path):
            print(f"Pose video not found: {pose_video_path}")
            continue
        
        # Use avatar image directly without cropping
        source_img = avatar_path
        
        # Run EDTalk inference with pose
        print(f"Processing: {output_filename}")
        print(f"Avatar: {os.path.basename(avatar_path)}")
        print(f"Audio: {os.path.basename(audio_path)}")
        print(f"Pose video: {os.path.basename(pose_video_path)}")
        
        cmd = f'python demo_lip_pose.py --source_path "{source_img}" --audio_driving_path "{audio_path}" --pose_driving_path "{pose_video_path}" --save_path "{output_path}" --face_sr'
        result = subprocess.run(cmd, shell=True, capture_output=True, cwd=EDTALK_DIR)
        
        success = result.returncode == 0
        
        if not success:
            print(f"Error output: {result.stderr.decode() if result.stderr else 'No error message'}")
            print(f"Stdout: {result.stdout.decode() if result.stdout else 'No stdout'}")
        
        if success:
            print(f"✓ Generated: {output_path}")
        else:
            print(f"✗ Failed: {output_path}")

# Set random seed for reproducible avatar selection
random.seed(42)

##### Fixed Pose

In [2]:
print("=== Processing Avatar Images with Fixed Pose ===")
scan_and_process_avatar_files()

=== Processing Avatar Images with Fixed Pose ===
Selected avatar for A010: A010_avatar_02.png
Selected avatar for A017: A017_avatar_01.png
Selected avatar for A014: A014_avatar_01.png
Selected avatar for A016: A016_avatar_03.png
Selected avatar for A002: A002_avatar_03.png
Selected avatar for A008: A008_avatar_03.png
Selected avatar for A011: A011_avatar_03.png
Selected avatar for A015: A015_avatar_01.png
Selected avatar for A006: A006_avatar_01.png
Selected avatar for S001: S001_avatar_03.png
Selected avatar for S012: S012_avatar_00.png
Selected avatar for S006: S006_avatar_00.png
Selected avatar for S008: S008_avatar_03.png
Selected avatar for S002: S002_avatar_03.png
Selected avatar for S011: S011_avatar_00.png
Selected avatar for S005: S005_avatar_00.png
Selected avatar for S007: S007_avatar_01.png
Selected avatar for S013: S013_avatar_00.png
Selected avatar for S003: S003_avatar_03.png
Selected avatar for S009: S009_avatar_00.png
Found 120 avatar-audio pairs to process


Processing all avatar tasks:   0%|          | 0/120 [00:00<?, ?it/s]

Processing: A010_avatar_02_CV_A010_BBP_cloned.mp4
Avatar: A010_avatar_02.png
Audio: CV_A010_BBP_cloned.wav


Processing all avatar tasks:   1%|          | 1/120 [01:12<2:24:21, 72.79s/it]

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_raw/avatar/RAW/A010_avatar_02_CV_A010_BBP_cloned.mp4
Processing: A010_avatar_02_XT_A010_BBP_cloned.mp4
Avatar: A010_avatar_02.png
Audio: XT_A010_BBP_cloned.wav


...

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_vctk/avatar/VCTK/S009_avatar_00_CV_S009_cloned.mp4
Processing: S009_avatar_00_XT_S009_cloned.mp4
Avatar: S009_avatar_00.png
Audio: XT_S009_VCTK_cloned.wav


Processing all avatar tasks:  99%|█████████▉| 119/120 [2:38:41<01:20, 80.53s/it]

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_vctk/avatar/VCTK/S009_avatar_00_XT_S009_cloned.mp4
Processing: S009_avatar_00_YT_S009_cloned.mp4
Avatar: S009_avatar_00.png
Audio: YT_S009_VCTK_cloned.wav


Processing all avatar tasks: 100%|██████████| 120/120 [2:39:54<00:00, 79.96s/it]

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_vctk/avatar/VCTK/S009_avatar_00_YT_S009_cloned.mp4


In [3]:
print("\n=== Processing Avatar Images with Original Video Pose ===")
scan_and_process_avatar_files_with_pose()


=== Processing Avatar Images with Original Video Pose ===
Selected avatar for A010: A010_avatar_02.png
Selected avatar for A017: A017_avatar_00.png
Selected avatar for A014: A014_avatar_00.png
Selected avatar for A016: A016_avatar_00.png
Selected avatar for A002: A002_avatar_00.png
Selected avatar for A008: A008_avatar_03.png
Selected avatar for A011: A011_avatar_02.png
Selected avatar for A015: A015_avatar_03.png
Selected avatar for A006: A006_avatar_02.png
Selected avatar for S001: S001_avatar_03.png
Selected avatar for S012: S012_avatar_03.png
Selected avatar for S006: S006_avatar_00.png
Selected avatar for S008: S008_avatar_00.png
Selected avatar for S002: S002_avatar_00.png
Selected avatar for S011: S011_avatar_02.png
Selected avatar for S005: S005_avatar_01.png
Selected avatar for S007: S007_avatar_01.png
Selected avatar for S013: S013_avatar_01.png
Selected avatar for S003: S003_avatar_03.png
Selected avatar for S009: S009_avatar_02.png
Found 120 avatar-audio-pose pairs to proc

Processing all avatar pose tasks:   0%|          | 0/120 [00:00<?, ?it/s]

Processing: A010_avatar_02_CV_A010_BBP_cloned.mp4
Avatar: A010_avatar_02.png
Audio: CV_A010_BBP_cloned.wav
Pose video: A010_02_BBP_NORMAL_color_25fps_6sec.mp4


Processing all avatar pose tasks:   1%|          | 1/120 [01:43<3:25:36, 103.67s/it]

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_raw/avatar_pose/RAW/A010_avatar_02_CV_A010_BBP_cloned.mp4
Processing: A010_avatar_02_XT_A010_BBP_cloned.mp4
Avatar: A010_avatar_02.png
Audio: XT_A010_BBP_cloned.wav
Pose video: A010_02_BBP_NORMAL_color_25fps_6sec.mp4


...

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_vctk/avatar_pose/VCTK/S009_avatar_02_CV_S009_cloned.mp4
Processing: S009_avatar_02_XT_S009_cloned.mp4
Avatar: S009_avatar_02.png
Audio: XT_S009_VCTK_cloned.wav
Pose video: S009_02_BBP_NORMAL_color_25fps_6sec.mp4


Processing all avatar pose tasks:  99%|█████████▉| 119/120 [2:37:39<01:05, 65.38s/it]

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_vctk/avatar_pose/VCTK/S009_avatar_02_XT_S009_cloned.mp4
Processing: S009_avatar_02_YT_S009_cloned.mp4
Avatar: S009_avatar_02.png
Audio: YT_S009_VCTK_cloned.wav
Pose video: S009_02_BBP_NORMAL_color_25fps_6sec.mp4


Processing all avatar pose tasks: 100%|██████████| 120/120 [2:38:44<00:00, 79.37s/it]

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_vctk/avatar_pose/VCTK/S009_avatar_02_YT_S009_cloned.mp4
